In [ ]:
# PARTIE 1 : INITIALISATION DU PROJET
# Charger les bibliothèques et le dataset RetailRocket

import sys
print('Python version: ' + str(sys.version))

import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
print("Bibliothèques importées avec succès !")

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
# PARTIE 1 : CHARGER LE DATASET

import os
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_dir = os.path.dirname(notebook_dir)
data_path = os.path.join(project_dir, 'data', 'events.csv')

if os.path.exists(data_path):
    df = pd.read_csv(data_path)
    print("OK - Dataset brut charge")
else:
    raise FileNotFoundError(f"{data_path} non trouve.")

print("\nPARTIE 1 : INITIALISATION DU PROJET")

# Informations basiques sur le dataset
print(f"\nDimension du dataset:")
print(f"  Enregistrements: {len(df):,}")
print(f"  Colonnes: {df.shape[1]}")
print(f"  Colonnes: {df.columns.tolist()}")

print(f"\nTypes de donnees:")
print(df.dtypes)

print(f"\nApercu:")
print(df.head(10))

# Verifications
assert not df.empty, "Le dataset est vide"
assert 'event' in df.columns, "Colonne 'event' manquante"
assert 'visitorid' in df.columns, "Colonne 'visitorid' manquante"
print("OK - Colonnes requises presentes")

# Repartition des evenements
event_counts = df['event'].value_counts()
print(f"\nTypes d'evenements:")
for event_type in event_counts.index:
    count = event_counts[event_type]
    prop = (100 * count / len(df))
    print(f"  {event_type}: {count:,} ({prop:.2f}%)")

In [ ]:
# PARTIE 2 : NETTOYAGE & EXPLORATION
# Conserver view et addtocart, vérifier les données manquantes

print("\nPARTIE 2 : NETTOYAGE & EXPLORATION")

# Filtrer uniquement view et addtocart
df_clean = df[df['event'].isin(['view', 'addtocart'])].copy()
print(f"\nFiltrage des evenements:")
print(f"  Avant: {len(df):,} enregistrements")
print(f"  Apres: {len(df_clean):,} enregistrements")
print(f"  Supprimes: {len(df) - len(df_clean):,}")

# Verifier les valeurs manquantes
print(f"\nValeurs manquantes:")
missing = df_clean.isnull().sum()
print(missing)

# Verifier colonnes critiques
if df_clean[['visitorid', 'event', 'timestamp']].isnull().sum().sum() > 0:
    df_clean = df_clean.dropna(subset=['visitorid', 'event', 'timestamp'])
    print(f"  Lignes supprimees: {len(df) - len(df_clean):,}")
else:
    print("  OK - Aucune valeur manquante critique")

# Visiteurs uniques
visiteurs_uniques = df_clean['visitorid'].nunique()
print(f"\nVisiteurs uniques: {visiteurs_uniques:,}")

# Distribution apres nettoyage
events_counts = df_clean['event'].value_counts()
print(f"\nDistribution apres nettoyage:")
for event in events_counts.index:
    count = events_counts[event]
    prop = (100 * count / len(df_clean))
    print(f"  {event}: {count:,} ({prop:.2f}%)")

# Reinitialiser l'index
df_clean.reset_index(drop=True, inplace=True)
print(f"\nOK - Dataset nettoye et pret")

In [ ]:
# PARTIE 3 : SIMULATION A/B TEST
# Attribuer aléatoirement chaque visiteur au groupe A ou B (50/50)
# Important: Randomisation par utilisateur, pas par événement

print("\nPARTIE 3 : SIMULATION A/B TEST")

# Créer liste des visiteurs uniques
visiteurs_df = pd.DataFrame({'visitorid': df_clean['visitorid'].unique()})

# Attribuer aléatoirement A ou B (50/50)
np.random.seed(42)
visiteurs_df['group'] = np.random.choice(['A', 'B'], size=len(visiteurs_df), p=[0.5, 0.5])

# Merger les groupes avec le dataset
df_clean = df_clean.merge(visiteurs_df, on='visitorid', how='left')

# Verifier la repartition
repartition = df_clean.groupby('visitorid')['group'].first().value_counts(normalize=True)
print(f"\nRepartition des groupes (par visiteur):")
print(f"  {repartition.to_dict()}")
print(f"OK - Randomisation 50/50 completee")

In [ ]:
# PARTIE 4 : CALCUL DU KPI (TAUX D'AJOUT AU PANIER)

def calculer_taux_basket(group_data):
    vues = (group_data['event'] == 'view').sum()
    baskets = (group_data['event'] == 'addtocart').sum()
    taux = baskets / vues if vues > 0 else 0.0
    return vues, baskets, taux

groupe_a_data = df_clean[df_clean['group'] == 'A']
groupe_b_data = df_clean[df_clean['group'] == 'B']

vues_a, baskets_a, taux_a = calculer_taux_basket(groupe_a_data)
vues_b, baskets_b, taux_b = calculer_taux_basket(groupe_b_data)

# Aliases pour coherence avec le test statistique
ajouts_a = baskets_a
ajouts_b = baskets_b

print("\nPARTIE 4 : KPI - TAUX D'AJOUT AU PANIER")
print(f"\nResultats du KPI:")
print(f"Groupe A: Vues={vues_a}, Ajouts={ajouts_a}, Taux={taux_a:.4f}")
print(f"Groupe B: Vues={vues_b}, Ajouts={ajouts_b}, Taux={taux_b:.4f}")
print(f"Difference B - A: {taux_b - taux_a:.4f}")

if taux_b > taux_a:
    print("Le groupe B performe mieux.")
elif taux_b < taux_a:
    print("Le groupe A performe mieux.")
else:
    print("Performances identiques.")

In [ ]:
# PARTIE 5 : TEST STATISTIQUE - TEST DE PROPORTION
# Z-test de comparaison des taux entre deux groupes
# H0: Les taux sont identiques
# H1: Les taux sont différents (test bilatéral)

print("\nPARTIE 5 : TEST STATISTIQUE - TEST DE PROPORTION")

print(f"\nHypotheses:")
print(f"  H0: Les taux de conversion A et B sont identiques")
print(f"  H1: Les taux de conversion A et B sont differents")
print(f"  Seuil α: 0.05")

# Donnees pour le test
succes_a = ajouts_a
n_a = vues_a
succes_b = ajouts_b
n_b = vues_b

print(f"\nDonnees du test:")
print(f"  Groupe A: succes={succes_a:,}, n={n_a:,}")
print(f"  Groupe B: succes={succes_b:,}, n={n_b:,}")

# Verifier les donnees
if n_a == 0 or n_b == 0:
    print("ERREUR: Donnees insuffisantes")
else:
    # Proportions individuelles
    prop_a = succes_a / n_a
    prop_b = succes_b / n_b
    
    print(f"\n1. Proportions:")
    print(f"  A: {prop_a:.6f} ({prop_a*100:.4f}%)")
    print(f"  B: {prop_b:.6f} ({prop_b*100:.4f}%)")
    print(f"  Difference: {(prop_b-prop_a)*100:+.4f}%")
    
    # Proportion combinee (pooled)
    p_pooled = (succes_a + succes_b) / (n_a + n_b)
    print(f"\n2. Proportion combinee (pooled): {p_pooled:.6f} ({p_pooled*100:.4f}%)")
    
    # Z-test
    z_stat, p_value = proportions_ztest(
        count=[succes_a, succes_b],
        nobs=[n_a, n_b]
    )
    
    print(f"\n3. Resultats du test:")
    print(f"  Z-statistic: {z_stat:.6f}")
    print(f"  p-valeur: {p_value:.6f}")
    
    # Conclusion
    alpha = 0.05
    print(f"\n4. Decision (α = {alpha}):")
    
    if p_value < alpha:
        print(f"  p-valeur ({p_value:.6f}) < α ({alpha})")
        print(f"  RESULTAT: SIGNIFICATIF")
        print(f"  Les taux sont DIFFERENTS")
    else:
        print(f"  p-valeur ({p_value:.6f}) > α ({alpha})")
        print(f"  RESULTAT: NON SIGNIFICATIF")
        print(f"  Pas de difference significative")
    
    # Performance
    print(f"\n5. Performance comparative:")
    if prop_b > prop_a:
        lift = ((prop_b - prop_a) / prop_a) * 100
        print(f"  Groupe B meilleur (lift: +{lift:.2f}%)")
    elif prop_a > prop_b:
        lift = ((prop_a - prop_b) / prop_b) * 100
        print(f"  Groupe A meilleur (lift: +{lift:.2f}%)")
    else:
        print(f"  Performances identiques")

In [ ]:
# PARTIE 6 : ANALYSE BUSINESS & RECOMMANDATION
# Interpréter les résultats et donner une recommandation

print("\nPARTIE 6 : ANALYSE BUSINESS & RECOMMANDATION")

# 1. Analyser la magnitude de la différence
print(f"\n1. Magnitude de la différence:")
diff = abs((prop_b - prop_a) * 100)
print(f"  Difference absolue: {diff:.4f}%")
print(f"  Difference relative: {abs((prop_b-prop_a)/prop_a)*100:.2f}%")

if diff < 0.1:
    magnitude = "NEGLIGEABLE"
elif diff < 0.5:
    magnitude = "FAIBLE"
elif diff < 1:
    magnitude = "MODEREE"
else:
    magnitude = "IMPORTANTE"
print(f"  Classification: {magnitude}")

# 2. Significativité statistique
print(f"\n2. Significativite statistique:")
print(f"  p-valeur: {p_value:.6f}")
print(f"  Seuil α: {alpha}")
sig = "SIGNIFICATIF" if p_value < alpha else "NON SIGNIFICATIF"
print(f"  Resultat: {sig}")

# 3. Risques d'erreur
print(f"\n3. Risques d'erreur:")
print(f"  Type I (Faux positif): {alpha*100}%")
print(f"  Type II (Faux negatif): ~20%")
print(f"  Probabilite hasard: {p_value*100:.2f}%")

# 4. Recommandation
print(f"\n4. RECOMMANDATION:")

if p_value < alpha:
    if diff > 0.5:
        rec = "DEPLOYER LE GROUPE B"
        raison = "Difference significative et suffisamment grande"
    else:
        rec = "CONTINUER LE TEST"
        raison = "Difference significative mais tres petite"
else:
    if diff > 0.5:
        rec = "CONTINUER LE TEST"
        raison = "Difference importante mais pas significative"
    else:
        rec = "GARDER LE GROUPE A (STATU QUO)"
        raison = "Pas de difference significative + tres petite"

print(f"  {rec}")
print(f"  Raison: {raison}")

# 5. Actions
print(f"\n5. Actions recommandees:")
if p_value < alpha:
    actions = [
        "Deployer le groupe gagnant",
        "Documenter les changements",
        "Monitoring post-deploiement",
        "Tester d'autres variantes"
    ]
else:
    actions = [
        "Maintenir le traitement actuel",
        "Analyser les retours utilisateurs",
        "Concevoir une variante C",
        "Planifier prochain test A/B"
    ]

for i, action in enumerate(actions, 1):
    print(f"  {i}. {action}")

In [ ]:
# PARTIE 7 : VISUALISATIONS & GRAPHIQUES
# Créer des graphiques pour visualiser les résultats

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

print("\nPARTIE 7 : VISUALISATIONS & GRAPHIQUES")

# Configuration
sns.set_style("whitegrid")
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

fig.suptitle('Analyse A/B Testing - RetailRocket (Complète)', fontsize=18, fontweight='bold', y=0.995)

# GRAPHIQUE 1: Comparaison des taux
ax1 = fig.add_subplot(gs[0, 0])
taux_values = [prop_a*100, prop_b*100]
colors = ['#1f4788', '#FF6B6B']
bars = ax1.bar(['Groupe A', 'Groupe B'], taux_values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('Taux (%)', fontweight='bold')
ax1.set_title('1. Comparaison des taux d\'ajout au panier', fontweight='bold')
ax1.set_ylim([2.4, 2.8])
ax1.grid(axis='y', alpha=0.3)

for bar, val in zip(bars, taux_values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
             f'{val:.4f}%', ha='center', va='bottom', fontweight='bold')

# GRAPHIQUE 2: Volume d'événements
ax2 = fig.add_subplot(gs[0, 1])
metrics = ['Vues', 'Add-to-Cart']
x = np.arange(len(metrics))
width = 0.35

vals_a = [vues_a/1000, ajouts_a/1000]
vals_b = [vues_b/1000, ajouts_b/1000]

ax2.bar(x - width/2, vals_a, width, label='Groupe A', color='#1f4788', alpha=0.8, edgecolor='black')
ax2.bar(x + width/2, vals_b, width, label='Groupe B', color='#FF6B6B', alpha=0.8, edgecolor='black')
ax2.set_ylabel('Événements (milliers)', fontweight='bold')
ax2.set_title('2. Volume d\'événements par groupe', fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(metrics)
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# GRAPHIQUE 3: Distribution View vs Add-to-Cart (Pie chart)
ax3 = fig.add_subplot(gs[1, 0])
sizes_a = [vues_a - ajouts_a, ajouts_a]
labels = ['View uniquement', 'Add-to-Cart']
colors_pie = ['#A8D8D8', '#FF6B6B']
ax3.pie(sizes_a, labels=labels, autopct='%1.1f%%', colors=colors_pie, startangle=90, 
        textprops={'fontweight': 'bold'})
ax3.set_title('3. Distribution Groupe A', fontweight='bold')

# GRAPHIQUE 4: Distribution View vs Add-to-Cart Groupe B
ax4 = fig.add_subplot(gs[1, 1])
sizes_b = [vues_b - ajouts_b, ajouts_b]
ax4.pie(sizes_b, labels=labels, autopct='%1.1f%%', colors=colors_pie, startangle=90,
        textprops={'fontweight': 'bold'})
ax4.set_title('4. Distribution Groupe B', fontweight='bold')

# GRAPHIQUE 5: Intervalles de confiance (95%)
ax5 = fig.add_subplot(gs[2, 0])

# Calcul des intervalles de confiance
z_alpha = 1.96  # 95% confiance
se_a = np.sqrt((prop_a * (1 - prop_a)) / n_a)
se_b = np.sqrt((prop_b * (1 - prop_b)) / n_b)

ci_a_lower = prop_a - z_alpha * se_a
ci_a_upper = prop_a + z_alpha * se_a
ci_b_lower = prop_b - z_alpha * se_b
ci_b_upper = prop_b + z_alpha * se_b

groups = ['Groupe A', 'Groupe B']
means = [prop_a*100, prop_b*100]
errors = [(ci_a_upper - ci_a_lower)*100/2, (ci_b_upper - ci_b_lower)*100/2]

ax5.bar(groups, means, yerr=errors, capsize=10, color=colors, alpha=0.8, 
        edgecolor='black', linewidth=1.5, error_kw={'linewidth': 2})
ax5.set_ylabel('Taux (%)', fontweight='bold')
ax5.set_title('5. Taux avec Intervalles de Confiance (95%)', fontweight='bold')
ax5.set_ylim([2.3, 2.9])
ax5.grid(axis='y', alpha=0.3)

for i, (mean, err) in enumerate(zip(means, errors)):
    ax5.text(i, mean + err + 0.05, f'{mean:.4f}%', ha='center', fontweight='bold')

# GRAPHIQUE 6: Tableau de synthèse
ax6 = fig.add_subplot(gs[2, 1])
ax6.axis('off')

summary_text = f"""
TABLEAU DE SYNTHESE

Métrique                    Groupe A        Groupe B        Différence
─────────────────────────────────────────────────────────────────────
Nombre de vues              {vues_a:>10,}     {vues_b:>10,}     {vues_b-vues_a:>+10,}
Ajouts au panier            {ajouts_a:>10,}     {ajouts_b:>10,}     {ajouts_b-ajouts_a:>+10,}
Taux (%)                    {prop_a*100:>10.4f}     {prop_b*100:>10.4f}     {(prop_b-prop_a)*100:>+10.4f}

TEST STATISTIQUE
─────────────────────────────────────────────────────────────────────
Z-statistic:                {z_stat:>10.6f}
p-valeur:                   {p_value:>10.6f}
Résultat:                   NON SIGNIFICATIF (p > 0.05)
IC 95% A:                   [{ci_a_lower*100:.4f}% - {ci_a_upper*100:.4f}%]
IC 95% B:                   [{ci_b_lower*100:.4f}% - {ci_b_upper*100:.4f}%]

DECISION: JE GARDE GROUPE A
"""

ax6.text(0.05, 0.95, summary_text, transform=ax6.transAxes, fontsize=9.5,
         verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.7, pad=1))

plt.savefig('Graphiques_AB_Testing.png', dpi=300, bbox_inches='tight')
print("\nOK - Graphiques sauvegares: Graphiques_AB_Testing.png")
plt.show()

print("\nOK - ANALYSE COMPLETE TERMINEE")